## Qiskit for k-coloring, NK method

from paper: A general quantum method to solve the graph K-colouring problem https://hal.science/hal-02891847v1

In [1]:
from qiskit import *
from qiskit_aer import QasmSimulator, AerError
from qiskit import QuantumCircuit, transpile
from qiskit.visualization import plot_histogram
import numpy as np
%matplotlib inline

In [3]:
# k-coloring of a graph
# N = number of nodes
# K = number of colours

nNodes = 3
nColors = 3  # nColors <= nNodes
nc = nColors + 1 
nn2 = round((nNodes-1)*nNodes/2)
sc = round(nc*nNodes)
sg = round(nc*nNodes + nn2)

nqubits = sc + 2*nn2 # Total number of qubits

# Creatr a Quantum Circuit
q = QuantumRegister(nqubits)
c = ClassicalRegister(nColors * nNodes)
qc = QuantumCircuit(q,c)
simulator = QasmSimulator()

qc.x(sg) # Set to |1> iif there is an edge
qc.x(sg+1)
qc.x(sg+2)
# qc.x(sg+3)
# qc.x(sg+4)
# qc.x(sg+5)

# Initialisation
s = 0
for n in range(nNodes):
    for k in range(nColors):
        qc.h(s+k)
    s = s + nc

# Constraints
s = 0
for n in range(nNodes):
    for k in range(nColors-1):
        for l in range(k+1, nColors):
            # Eliminate 11
            qc.ccx(s+k, s+l, s+nColors)
            qc.cx(s+nColors, s+k)
            qc.reset(s+nColors)

        # Eliminate 0* (no color assigned to the node n)
    for k in range(nColors):
        qc.x(s+k)
        cb = list(range(s, s+nColors))
        qc.mcx(cb,s+nColors)

    for k in range(nColors):
        qc.x(s+k)
        qc.cx(s+nColors, s+nColors-1)
        qc.reset(s+nColors)
    s = s + nc

print('end of colouring matrices')

for k in range(nColors):
    s = nc*nNodes
    for n1 in range(nNodes-1):
        for n2 in range(n1+1,nNodes):
            n11 = nc*n1+k
            n22 = nc*n2+k
            qc.ccx(n11,n22,s)
            s = s+1

print('end of pairs of nodes')


end of colouring matrices
end of pairs of nodes


In [4]:
# Compare to the graph
for n in range(sc,sc+nn2):
    for node in range(nNodes):
        qnode = nc*node
        qnc = qnode+nColors
        for k in range(nColors):
            cb = [n, n+nn2, qnode+k]
            qc.mcx(cb,qnc)
            cb = [n, n+nn2, qnc]
            qc.mcx(cb, qnode+k)
        qc.reset(qnc)
    print('end of compare to graph')

end of compare to graph
end of compare to graph
end of compare to graph


In [5]:
# Measure
cb = 0
for n in range(nNodes):
    s = n*(nColors+1)
    for k in range(nColors):
        qb = s+k
        qc.measure(qb,cb)
        cb = cb +1
print('end of measure')

end of measure


In [6]:
# Execute the circuit on the qasm simulator
compiled_circuit = transpile(qc, simulator)
job = simulator.run(compiled_circuit, shots = 1000)
# job = execute(qc, simulator, shots=1000)

# Grab the results from the job
result = job.result()
print('end of execute')

counts = result.get_counts(compiled_circuit)
print(counts)
print('The solutions are giver by the strings that are not 0*')
print('Please check: some of them may need LESS than', nColors, 'colours')
qc.draw()


end of execute
{'100000001': 9, '000000000': 158, '000010001': 20, '100001010': 3, '000100000': 45, '010100001': 1, '000001000': 28, '100100000': 41, '110000110': 7, '100010000': 16, '000100010': 15, '000100001': 4, '000001010': 20, '000110110': 5, '000000001': 35, '100000000': 30, '100000100': 47, '001000100': 6, '000000010': 65, '110100110': 5, '001000010': 17, '000000100': 31, '110110000': 13, '000001100': 7, '100010001': 6, '000100100': 36, '001010100': 4, '010001100': 4, '010000000': 66, '000010000': 56, '001000000': 36, '110100100': 8, '000000110': 2, '001010000': 19, '100110100': 15, '001100000': 9, '100001000': 6, '010100000': 21, '010001000': 13, '000010100': 10, '100000010': 10, '010000001': 18, '100100110': 6, '110000000': 3, '110110100': 4, '000110000': 2, '100110110': 3, '010000100': 11, '001100010': 4}
The solutions are giver by the strings that are not 0*
Please check: some of them may need LESS than 3 colours


┌───┐     ┌───┐          ┌───┐┌───┐                                   »
 q2_0: ┤ H ├──■──┤ X ├───────■──┤ X ├┤ X ├─────────────────■─────────■───────»
       ├───┤  │  └─┬─┘       │  └─┬─┘└───┘     ┌───┐       │  ┌───┐  │       »
 q2_1: ┤ H ├──■────┼─────────┼────┼─────────■──┤ X ├───────■──┤ X ├──■───────»
       ├───┤  │    │         │    │         │  └─┬─┘       │  └───┘  │  ┌───┐»
 q2_2: ┤ H ├──┼────┼─────────■────┼─────────■────┼─────────■─────────■──┤ X ├»
       └───┘┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐     ┌─┴─┐└───┘»
 q2_3: ─────┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├─────┤ X ├─────»
       ┌───┐└───┘┌───┐     └───┘┌───┐┌───┐└───┘          └───┘     └───┘     »
 q2_4: ┤ H ├──■──┤ X ├───────■──┤ X ├┤ X ├─────────────────■─────────■───────»
       ├───┤  │  └─┬─┘       │  └─┬─┘└───┘     ┌───┐       │  ┌───┐  │       »
 q2_5: ┤ H ├──■────┼─────────┼────┼─────────■──┤ X ├───────■──┤ X ├──■───────»
       ├───┤  │    │         │    │         │  └─┬─┘       │  └───┘  │  ┌───┐»
 q2_6: ┤ H ├──┼────┼─────────■────┼─────────■────┼─────────■─────────■──┤ X ├»
       └───┘┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐     ┌─┴─┐└───┘»
 q2_7: ─────┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├─────┤ X ├─────»
       ┌───┐└───┘┌───┐     └───┘┌───┐┌───┐└───┘          └───┘     └───┘     »
 q2_8: ┤ H ├──■──┤ X ├───────■──┤ X ├┤ X ├─────────────────■─────────■───────»
       ├───┤  │  └─┬─┘       │  └─┬─┘└───┘     ┌───┐       │  ┌───┐  │       »
 q2_9: ┤ H ├──■────┼─────────┼────┼─────────■──┤ X ├───────■──┤ X ├──■───────»
       ├───┤  │    │         │    │         │  └─┬─┘       │  └───┘  │  ┌───┐»
q2_10: ┤ H ├──┼────┼─────────■────┼─────────■────┼─────────■─────────■──┤ X ├»
       └───┘┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐  │       ┌─┴─┐     ┌─┴─┐└───┘»
q2_11: ─────┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├──■───|0>─┤ X ├─────┤ X ├─────»
            └───┘          └───┘          └───┘          └───┘     └───┘     »
q2_12: ──────────────────────────────────────────────────────────────────────»
                                                                             »
q2_13: ──────────────────────────────────────────────────────────────────────»
                                                                             »
q2_14: ──────────────────────────────────────────────────────────────────────»
       ┌───┐                                                                 »
q2_15: ┤ X ├─────────────────────────────────────────────────────────────────»
       ├───┤                                                                 »
q2_16: ┤ X ├─────────────────────────────────────────────────────────────────»
       ├───┤                                                                 »
q2_17: ┤ X ├─────────────────────────────────────────────────────────────────»
       └───┘                                                                 »
 c1: 9/══════════════════════════════════════════════════════════════════════»
                                                                             »
«            ┌───┐                                                            »
« q2_0: ──■──┤ X ├───────■─────────■──────────────────────────────────────────»
«         │  ├───┤       │         │                                          »
« q2_1: ──■──┤ X ├───────┼─────────┼────■──────────────■──────────────────────»
«         │  ├───┤       │  ┌───┐  │    │  ┌───┐┌───┐  │                      »
« q2_2: ──■──┤ X ├───────┼──┤ X ├──┼────┼──┤ X ├┤ X ├──┼───────────────────■──»
«       ┌─┴─┐└─┬─┘       │  └─┬─┘  │    │  └───┘└─┬─┘  │                   │  »
« q2_3: ┤ X ├──■───|0>───┼────■────┼────┼───|0>───■────┼───|0>─────────────┼──»
«       └───┘┌───┐       │         │    │              │                   │  »
« q2_4: ──■──┤ X ├───────■─────────┼────┼─────────■────┼───────────────────┼──»
«         │  ├───┤       │         │    │         │    │                   │  »
« q2_5: ──■──┤ X ├───────┼─────────┼────■─────

In [10]:
sorted_items = sorted(counts.items(), key=lambda x: x[1], reverse=True)

for i in range(15):
    print(sorted_items[i])

('000000000', 158)
('010000000', 66)
('000000010', 65)
('000010000', 56)
('100000100', 47)
('000100000', 45)
('100100000', 41)
('000100100', 36)
('001000000', 36)
('000000001', 35)
('000000100', 31)
('100000000', 30)
('000001000', 28)
('010100000', 21)
('000010001', 20)
